In [ ]:
!pip install pytranskit


In [ ]:
import cv2
import numpy as np
from PIL import Image
import pickle
import pytranskit
from pytranskit.optrans.continuous.radoncdt import RadonCDT
from google.colab.patches import cv2_imshow  # Import for displaying images in Colab

# Load trained model
model_path = "/content/Rrcdt_ns_model (25).pkl"
with open(model_path, "rb") as f:
    classifier = pickle.load(f)

theta = np.linspace(0, 176, 45)  # Ensure it matches training
radoncdt = RadonCDT(theta)

def preprocess_image(image, img_size=28):
    """Preprocess a digit image for prediction"""
    img = Image.fromarray(image).convert("L")  # Convert to grayscale
    img = img.resize((img_size, img_size))  # Resize to match training data
    img_array = np.array(img, dtype=np.float32)

    # Ensure white numbers on black background
    if np.mean(img_array) > 127:  # If background is white, invert colors
        img_array = 255 - img_array

    img_array = img_array / 255.0  # Normalize pixel values

    # Ensure input is 3D (N, H, W) to match expected input
    img_array = img_array[np.newaxis, :, :]  # Convert (H, W) -> (1, H, W)

    template = np.ones(img_array[0].shape, dtype=img_array.dtype)

    # Apply Radon-CDT transformation
    img_rcdt = radoncdt.forward(
        [0, 1], template / np.sum(template),
        [0, 1], img_array[0] / np.sum(img_array[0]),
        True
    )

    # Return the transformed image
    img_rcdt = img_rcdt[np.newaxis, :, :]  # Ensure it's a 3D array (1, H, W)
    return img_array

def predict_digit(image):
    """Predict digit from preprocessed image"""
    processed_image = preprocess_image(image)
    prediction = classifier.predict(processed_image)  # Use trained classifier
    return prediction[0]

def extract_and_classify_numbers(image_path):
    """Detects numbers in an image, extracts and classifies using the trained model"""
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Threshold to get binary image
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours of numbers
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    digit_data = []

    # Iterate through contours, extract each digit image, and classify
    for idx, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)
        if w * h > 50:  # Filter noise by contour area
            number_img = gray[y:y + h, x:x + w]

            # Show the extracted digit image for debugging (using cv2_imshow in Colab)
            cv2_imshow(number_img)  # Display extracted digit image in Colab

            predicted_digit = predict_digit(number_img)  # Predict digit
            digit_data.append((x, predicted_digit))

            # Optionally save the digit image for review
            # digit_img = Image.fromarray(number_img)
            # digit_img.save(f"/content/drive/MyDrive/extracted_digits/digit_{idx}.png")  # Save image to Google Drive

    # Sort detected digits by x-coordinate (left to right)
    digit_data.sort()
    detected_numbers = [digit for _, digit in digit_data]
    print("Detected Numbers (In Order):", detected_numbers)

    return detected_numbers

# Example usage:
image_path = '/content/ret.png'  # Replace with the image you want to use
detected_numbers = extract_and_classify_numbers(image_path)
print("Detected Numbers:", detected_numbers)
